# Gráficas para estudio de reservas en Guatemala

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt 
import seaborn as sns
import numpy as np
from statsmodels.tsa.seasonal import seasonal_decompose
from statsmodels.tsa.arima.model import ARIMA
from statsmodels.tsa.statespace.sarimax import SARIMAX
from statsmodels.graphics.tsaplots import plot_acf, plot_pacf
from pmdarima.arima import auto_arima
import datetime
import matplotlib.dates as mdates

In [ ]:
reservas = pd.read_excel("db/datos_reservas.xlsx")

In [ ]:
reservas

In [ ]:
plt.figure(figsize=(15, 8))
sns.lineplot(data=reservas.melt('Mes', var_name='Año', value_name='Reservas'), x='Mes', y='Reservas', hue='Año', marker='o', palette=["#19315F", "#F48401", "#BDDB05", "#11C876", "#879484", "#BC2EE6", "#621A22"])

plt.xticks(rotation=45)
plt.title('Reservas Bancarias por Año')
plt.xlabel('Mes')
plt.ylabel('Reservas en millones de quetzales')

plt.legend(title='Año', loc='upper left', bbox_to_anchor=(1, 1))

plt.show()

## Calcular variaciones intermensuales e interanuales

In [ ]:
# Variaciones intermensuales
int_mensuales = reservas[[i for i in range(2018, 2022)]].diff().fillna(0)
int_mensuales

In [ ]:
# Hacer diccionario con estadísticos por año
est_anuales = {i: [0,0,0,0] for i in range(2018,2022)}
for i in range(2018, 2022):
    est_anuales[i][0] = reservas[i].mean()
    est_anuales[i][1] = reservas[i].var()
    est_anuales[i][2] = reservas[i].min()
    est_anuales[i][3] = reservas[i].max()

df_est_anuales = pd.DataFrame.from_dict(est_anuales, orient="index", columns=["Promedio", "Varianza", "Mínimo", "Máximo"])
df_est_anuales.index.name = "Año"

df_est_anuales

## Análisis de reservas e inflación

In [ ]:
reservas_iniciales_anuales = reservas[reservas["Mes"] == "Enero"]
ritmo_inflacionario = pd.read_excel("db/Ritmo_inflacionario.xlsx")

In [ ]:
ritmo_inflacionario

In [ ]:
dic_proyeccion = {i: [None]*12 for i in range(2018, 2022)}
dic_proyeccion["Mes"] = ritmo_inflacionario["Año"].tolist()
df_proyeccion = pd.DataFrame.from_dict(dic_proyeccion)
df_proyeccion = df_proyeccion[["Mes", 2018, 2019, 2020, 2021, 2022, 2023]]

In [ ]:
for i in range(2018, 2024):
    for j in range(0, 12):
        if j == 0:
            df_proyeccion.loc[0, i] = reservas.loc[0, i]
        else:
           df_proyeccion.loc[j, i] = df_proyeccion.loc[j-1, i] * (1 + ritmo_inflacionario.loc[j-1,i] / 100)

In [ ]:
df_proyeccion

In [ ]:
# Combinar dfs para poder graficar
df_combinado = reservas.join(df_proyeccion, lsuffix="_re", rsuffix="_pro")
plt.figure(figsize=(10, 12))

for year in range(2018, 2024):  
    plt.figure(figsize=(10, 5))  
    plt.plot(df_combinado["Mes_re"], df_combinado[str(year) + '_re'], marker='o', label='Reservas reportadas', color="#FF0000")
    plt.plot(df_combinado["Mes_re"], df_combinado[str(year) + '_pro'], marker='o', linestyle='--', label='Proyección con ritmo inflacionario', color="#0000ff")
    
    plt.title(f'Reservas Bancarias - {year}')
    plt.xlabel('Mes')
    plt.xticks(rotation=25)
    plt.ylabel('Reservas en millones de quetzales')
    plt.legend()
    plt.grid(True)
    
    plt.savefig(f'imagenes/reservas_{year}.png', bbox_inches="tight")
    plt.close()  

## Segundo Análsis de inflación

In [ ]:
dic_retro = {i: [None]*12 for i in range(2018, 2024)}
dic_retro["Mes"] = ritmo_inflacionario["Año"].tolist()
df_retro = pd.DataFrame.from_dict(dic_proyeccion)
df_retro = df_retro[["Mes", 2018, 2019, 2020, 2021, 2022, 2023]]

In [ ]:
for i in range(2018, 2024):
    for j in range(0, 12):
        if j == 0:
            df_retro.loc[0, i] = reservas.loc[0, i]
        else:
           df_retro.loc[j, i] = df_retro.loc[j-1, i] * (1 - ritmo_inflacionario.loc[j-1,i] / 100)
            

In [ ]:
# Combinar dfs para poder graficar
df_combinado_2 = reservas.join(df_retro, lsuffix="_re", rsuffix="_retro")
plt.figure(figsize=(10, 12))

for year in range(2018, 2024):  
    plt.figure(figsize=(10, 5))  
    plt.plot(df_combinado_2["Mes_re"], df_combinado_2[str(year) + '_re'], marker='o', label='Reservas reportadas', color="#FF0000")
    plt.plot(df_combinado_2["Mes_re"], df_combinado_2[str(year) + '_retro'], marker='o', linestyle='--', label='Proyección sin ritmo inflacionario', color="#0000ff")
    
    plt.title(f'Reservas Bancarias - {year}')
    plt.xlabel('Mes')
    plt.xticks(rotation=25)
    plt.ylabel('Reservas en millones de quetzales')
    plt.legend()
    plt.grid(True)
    
    plt.savefig(f'imagenes/retro_{year}.png', bbox_inches="tight")
    plt.close()  

## Modelo SARIMA

In [154]:
# Retomar los df de ritmo inflacionario y el de reservas para hacer arima
serie_inflacion = []
serie_inflacion_testeo = []
tiempo = []
tiempo_testeo = []

for i in range(2018, 2024):
    serie_inflacion += list(ritmo_inflacionario[i])

for i in range(2018, 2024):
    serie_inflacion_testeo += list(ritmo_inflacionario[i])

for i in range(2018, 2024):
    for j in range(1,10):
        tiempo.append(f"01/0{j}/{i}")
    for j in range(10, 13):
        tiempo.append(f"01/{j}/{i}")

for i in range(2018, 2024):
    for j in range(1,10):
        tiempo_testeo.append(f"01/0{j}/{i}")
    for j in range(10, 13):
        tiempo_testeo.append(f"01/{j}/{i}")

# Hacer df para arima con datos de inflacion 
dic_arima_inflacion = {"Tiempo": tiempo, "Inflación": serie_inflacion}
df_arima_inflacion = pd.DataFrame.from_dict(dic_arima_inflacion)
df_arima_inflacion["Tiempo"] = pd.to_datetime(df_arima_inflacion["Tiempo"])
df_arima_inflacion["Inflación"] = df_arima_inflacion["Inflación"].astype("float")
df_arima_inflacion.set_index("Tiempo", inplace=True)

# Hacer df de testeo
dic_arima_inflacion_testeo = {"Tiempo": tiempo_testeo, "Inflación": serie_inflacion_testeo}
df_arima_inflacion_testeo = pd.DataFrame.from_dict(dic_arima_inflacion_testeo)
df_arima_inflacion_testeo["Tiempo"] = pd.to_datetime(df_arima_inflacion_testeo["Tiempo"])
df_arima_inflacion_testeo["Inflación"] = df_arima_inflacion_testeo["Inflación"].astype("float")
df_arima_inflacion_testeo.set_index("Tiempo", inplace=True)

# Repetir para datos de reservas
serie_reservas = []
serie_reservas_testeo = []

for i in range(2018, 2024):
    serie_reservas += list(reservas[i])

for i in range(2018, 2024):
    serie_reservas_testeo += list(reservas[i])

# Hacer df para arima con datos de reservas 
dic_arima_reservas = {"Tiempo": tiempo, "Reservas": serie_reservas}
df_arima_reservas = pd.DataFrame.from_dict(dic_arima_reservas)
df_arima_reservas["Tiempo"] = pd.to_datetime(df_arima_reservas["Tiempo"])
df_arima_reservas["Reservas"] = df_arima_reservas["Reservas"].astype("float")
df_arima_reservas.set_index("Tiempo", inplace=True)

# Hacer df de testeo
dic_arima_reservas_testeo = {"Tiempo": tiempo_testeo, "Reservas": serie_reservas_testeo}
df_arima_reservas_testeo = pd.DataFrame.from_dict(dic_arima_reservas_testeo)
df_arima_reservas_testeo["Tiempo"] = pd.to_datetime(df_arima_reservas_testeo["Tiempo"])
df_arima_reservas_testeo["Reservas"] = df_arima_reservas_testeo["Reservas"].astype("float")
df_arima_reservas_testeo.set_index("Tiempo", inplace=True)

In [ ]:
def hallar_parametros_arima(df, variables):
    modelo_var = {var : (0,0,0) for var in variables}
    for var in variables:
        modelo = auto_arima(df[var], max_p=100, max_d=5, max_q=100)
        modelo_var[var] = modelo.order
    return modelo_var

In [ ]:
# # Hallar parámetros para arima
parametros_inflacion = hallar_parametros_arima(df_arima_inflacion, ["Inflación"])["Inflación"]
parametros_reservas = hallar_parametros_arima(df_arima_reservas, ["Reservas"])["Reservas"]

In [ ]:
# Aplicar SARIMA
model_sarima_reservas = SARIMAX(df_arima_reservas["Reservas"], order=parametros_reservas, seasonal_order=(0,0,2,12))
modelo_fit__sarima_reservas = model_sarima_reservas.fit()

# Realizar predicciones
predicciones_sarima_reservas = modelo_fit__sarima_reservas.get_forecast(steps=12)
predicciones_sarima_reservas = predicciones_sarima_reservas.predicted_mean

## Graficar datos y proyecciones

In [ ]:
plt.figure(figsize=(15, 8))
sns.lineplot(data=reservas.melt('Mes', var_name='Año', value_name='Reservas'), x='Mes', y='Reservas', hue='Año', marker='o', palette=["#19315F", "#F48401", "#BDDB05", "#11C876", "#879484", "#BC2EE6", "#621A22"])

plt.xticks(rotation=45)
plt.title('Reservas Bancarias por Año')
plt.xlabel('Mes')
plt.ylabel('Reservas en millones de quetzales')

plt.legend(title='Año', loc='upper left', bbox_to_anchor=(1, 1))

plt.savefig("imagenes/proyecciones_sarima/prediccion_reservas_sarima.png")
plt.show()

## Utilizar proyecciones para repetir gráficos del inicio

## Gráficas para SARIMA

In [164]:
from statsmodels.tools.eval_measures import rmse, meanabs
import numpy as np

# Para reservas
mse_reservas = np.mean((predicciones_sarima_reservas - serie_reservas[60:72])**2)
rmse_reservas = rmse(predicciones_sarima_reservas, serie_reservas[60:72])
mae_reservas = meanabs(predicciones_sarima_reservas, serie_reservas[60:72])

print(f"Reservas - MSE: {mse_reservas}, RMSE: {rmse_reservas}, MAE: {mae_reservas}")
print(f"Reservas - AIC: {modelo_fit__sarima_reservas.aic}, BIC: {modelo_fit__sarima_reservas.bic}")


Reservas - MSE: 3671571.7099791807, RMSE: 1916.1345751223166, MAE: 1558.965424806036
Reservas - AIC: 1136.7488587335279, BIC: 1147.1365459530564
